In [21]:
# 导入包，之前没用过baostock的先pip
import baostock as bs
# 数据处理包
import pandas as pd
import numpy as np
# 时间包
import datetime
# 系统路径包
import os
# 不显示警告
import warnings
warnings.filterwarnings('ignore')
# 连接baostock服务器
lg = bs.login()
# 获得2022-7-11当天所有股票名，测试一下是否连接上
rs = bs.query_all_stock(day="2022-07-11")
all_stocks = []
while (rs.error_code == '0') & rs.next():
    all_stocks.append(rs.get_row_data())
all_code = pd.DataFrame(all_stocks, columns=rs.fields)
all_code.head(5)

login success!


,code,tradeStatus,code_name
0,bj.430047,1,
1,bj.430090,1,
2,bj.430198,1,
3,bj.430418,1,
4,bj.430489,1,


In [18]:
# 逐行对每个股票进行处理，获取所需要的数据，这里下载的是每个股票的日线级别数据
for _,line in all_code.iterrows():
    # 如果股票代码里有BJ,不考虑，跳掉
    if line['code'][:2] == 'bj':
        continue
    # 如果股票代码里有SH.0,不考虑，跳掉
    if line['code'][:4] =='sh.0':
        continue
    # 如果股票的交易状态是1（没退市）
    if line['tradeStatus'] == '1' and not '指数' in line['code_name']:
        # 获取特定数据的K线,这里下载了2022-1-1到2022-11-30的A股数据
        rs = bs.query_history_k_data_plus(str(line['code']),"date,code,open,high,low,close,preclose,volume,amount,turn,pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM,isST",start_date='2022-01-01',end_date='2022-11-30',frequency="d", adjustflag="2")
        data_list = []
        while (rs.error_code == '0') & rs.next():
            data_list.append(rs.get_row_data())
        result = pd.DataFrame(data_list, columns=rs.fields)
        # 存入文件于次级目录raw中，生成的是csv格式文件
        try:
            if not result.empty:
                result.to_csv('D:\\data\\raw\\'+line['code']+'.csv',index=False)
        # 如果没存成功，则不管，打印一下warning
        except:
            print(line['code'],'warning')